In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import (DataLoader, Dataset)

import pickle
import sys
sys.path.append("../")

import torch
from torch.nn.utils.rnn import PackedSequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn as nn

# from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig, AutoModel

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from src.module.BaselineModule import BaselineModule
from src.module.model.TwoHeadLSTM import TwoHeadLSTM, elementwise_apply
from src.data.SeqData import SeqData


from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients, IntegratedGradients


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:572: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.object, string),
/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:573: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.bool, bool),
/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/tensorboar

In [92]:
with open("expl.pl", "rb") as f:
    records = pickle.load(f)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [109]:
from IPython.display import display

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [113]:
for i, record in enumerate(records):
    print("sample", i)
    viz.visualize_text(records[i])
print()

sample 0


/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


sample 1


sample 2


sample 3


...

sample 19


sample 20


sample 21


sample 22


sample 23


In [27]:
with open("subseq_occlusion.pl", "rb") as f:
    occlusion = pickle.load(f)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
occlusion[0][1]

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
  ['m', 'r', 'm', 'r', 'p', 'p', 'p', 'p', 'm', 'r'],
  -3.725290298461914e-09),
 (array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
  ['r', 'm', 'r', 'p', 'p', 'p', 'p', 'm', 'r', 'r'],
  -3.725290298461914e-09),
 (array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
  ['m', 'r', 'p', 'p', 'p', 'p', 'm', 'r', 'r', 'm'],
  -2.7939677238464355e-09),
 (array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
  ['r', 'p', 'p', 'p', 'p', 'm', 'r', 'r', 'm', 'p'],
  -3.725290298461914e-09),
 (array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
  ['p', 'p', 'p', 'p', 'm', 'r', 'r', 'm', 'p', 'p'],
  -3.725290298461914e-09),
 (array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
  ['p', 'p', 'p', 'm', 'r', 'r', 'm', 'p', 'p', 'r'],
  -5.587935447692871e-09),
 (array([ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
  ['p', 'p', 'm', 'r', 'r', 'm', 'p', 'p', 'r', 'c'],
  -3.725290298461914e-09),
 (array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16]),
  ['p', 'm', 'r', 'r', 'm', 'p', '

In [119]:
with open("occlusion.pl", "rb") as f:
    occlusion = pickle.load(f)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [129]:
for i in occlusion:
    L, R = occlusion[i]
    print("sample", i)
    for desc, LR in zip(["L", "R"], [L, R]):
        most_salient_keys = sorted(LR, key=lambda k: abs(LR[k][1]), reverse=True)[:10]
        print(desc)
        print("\t top 10 nonzero \t", ", ".join([
            f"{LR[k][0]} ({LR[k][1]})"
            for k in most_salient_keys
            if LR[k][1] != 0
        ]))
    print()

sample 0
L
	 top 10 nonzero 	 ['r', 'r', 'r'] (0.0022853445261716843), ['p', 'p', 'p'] (0.0012485701590776443), ['c', 'c', 'c'] (-2.296827733516693e-05), ['c', 'c', 'c', 'c', 'c'] (6.474554538726807e-06), ['c', 'c', 'c', 'c'] (-1.1175870895385742e-08), ['p', 'p', 'p'] (-4.6566128730773926e-09), ['c', 'c', 'c'] (-2.7939677238464355e-09), ['c', 'c', 'c'] (-2.7939677238464355e-09), ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'] (-2.7939677238464355e-09), ['p', 'p', 'p'] (-2.7939677238464355e-09)
R
	 top 10 nonzero 	 ['p', 'p', 'p', 'p'] (0.0024926532059907913), ['p', 'p', 'p', 'p', 'p', 'p'] (3.064516931772232e-05), ['c', 'c', 'c', 'c', 'c'] (-7.456168532371521e-06), ['c', 'c', 'c', 'c'] (-2.1420419216156006e-07), ['p', 'p', 'p', 'p'] (-4.6566128730773926e-09), ['p', 'p', 'p'] (-4.6566128730773926e-09), ['c', 'c', 'c'] (-4.6566128730773926e-09), ['c', 'c', 'c'] (-4.6566128730773926e-09), ['w', 'w', 'w'] (-3.725290298461914e-09), ['p', 'p', 'p', 'p', 'p', 'p'] (-3.725290298461914e-09)

sample 1


/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
